In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf

In [4]:
data=pd.read_csv('https://raw.githubusercontent.com//rajtilakls2510//car_price_predictor//master//quikr_car.csv')

In [5]:
data.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [6]:
data['year'].unique()

array(['2007', '2006', '2018', '2014', '2015', '2012', '2013', '2016',
       '2010', '2017', '2008', '2011', '2019', '2009', '2005', '2000',
       '...', '150k', 'TOUR', '2003', 'r 15', '2004', 'Zest', '/-Rs',
       'sale', '1995', 'ara)', '2002', 'SELL', '2001', 'tion', 'odel',
       '2 bs', 'arry', 'Eon', 'o...', 'ture', 'emi', 'car', 'able', 'no.',
       'd...', 'SALE', 'digo', 'sell', 'd Ex', 'n...', 'e...', 'D...',
       ', Ac', 'go .', 'k...', 'o c4', 'zire', 'cent', 'Sumo', 'cab',
       't xe', 'EV2', 'r...', 'zest'], dtype=object)

In [7]:
data.shape

(892, 6)

In [8]:
data=data[data['year'].str.isnumeric()]

In [9]:
data=data[data['Price']!="Ask For Price"]


In [10]:
data['Price']=data['Price'].str.replace(',','').astype(int)

In [11]:
data['year']=data['year'].astype(int)

In [12]:
data['kms_driven']

0        45,000 kms
1            40 kms
3        28,000 kms
4        36,000 kms
6        41,000 kms
           ...     
886    1,32,000 kms
888      27,000 kms
889      40,000 kms
890          Petrol
891          Petrol
Name: kms_driven, Length: 819, dtype: object

In [13]:
data['kms_driven']=data['kms_driven'].str.split().str.get(0).str.replace(',','')

In [14]:
data=data[data['kms_driven'].str.isnumeric()]

In [15]:
data['kms_driven']=data['kms_driven'].astype(int)

In [16]:
data['name']=data['name'].str.split().str.slice(start=0,stop=3).str.join(' ')

In [17]:
data.describe()

,year,Price,kms_driven
count,817.000000,8.170000e+02,817.000000
mean,2012.440636,4.115503e+05,46250.714810
std,4.002354,4.749173e+05,34283.745254
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.900000e+05,56758.000000
max,2019.000000,8.500003e+06,400000.000000


In [18]:
data=data[data['Price']<6e6].reset_index(drop=True)

In [19]:
data.shape

(816, 6)

In [20]:
data.to_csv('car_new.csv')

## Model

In [21]:
feature=data.drop(columns='Price')
target=data['Price']

In [22]:
from sklearn.model_selection import train_test_split


In [23]:
x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.2)

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [25]:
onehot=OneHotEncoder()
onehot.fit(feature[['name','company','fuel_type']])

OneHotEncoder()

In [26]:
column_trans=make_column_transformer((OneHotEncoder(categories=onehot.categories_),['name','company','fuel_type']),remainder='passthrough')

In [27]:
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)

In [28]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Pe

In [29]:
y_pred=pipe.predict(x_test)

In [30]:
r2_score(y_test,y_pred)

0.7722160871770609

In [31]:
rscore=[]
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    scr=r2_score(y_test,y_pred)
    rscore.append(scr)
    
    
    

In [32]:
import numpy as np
np.argmax(rscore)

651

In [33]:
rscore[np.argmax(rscore)]

-0.24705955481567954

In [34]:
x_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.2,random_state=np.argmax(rscore))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(x_train,y_train)
res=r2_score(y_test,y_pred)


In [35]:
res

-0.24705955481567954

In [36]:
import pickle

In [37]:
pickle.dump(pipe,open('regression.pkl','wb'))

In [41]:
predr=pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']],columns=['name','company','year','kms_driven','fuel_type']))

In [42]:
predr

array([424773.68231428])